# 作业一：布尔检索
### 实现及改进的点
- 利用pd.array().unique()来实现列表的去重
- 实现了多个and的优化

### 遇到的困难
- np.array()的自动类型转换会导致列表list连接运算符+变成列表项相加
- (based&&!day)||(based&&day)最外边的括号对符合语法，但是这里不需要去掉括号



In [215]:
import re
import os
import nltk
import numpy as np
import pandas as pd
from string import punctuation
from nltk.corpus import stopwords
from collections import defaultdict
sw = stopwords.words('english')

## 对文本进行分词
使用了NLTK工具

In [216]:
def get_words(text):
    text = re.sub(r"[{}]+".format(punctuation), " ", text)  # 将标点符号转化为空格
    text = text.lower()  # 全部字符转为小写
    words = nltk.word_tokenize(text)  # 分词
    words = list(set(words).difference(set(sw)))  # 去停用词
    return words

## 获取文本文件
给定文本文件目录，获取目录下所有符合要求的文件列表

In [217]:
def get_files(dir, file_type='.txt'):
    '''
    获取文件名列表
    :param dir: 
    :param file_type: 
    :return: 
    '''
    file_list = []
    for home, dirs, files in os.walk(dir):
        for filename in files:
            if file_type in filename:
                file_list.append(os.path.join(home, filename))
    return file_list

## 词法分析
通过正则表达式对查询进行词法分析

In [218]:
# 构造每种类型词的正则表达式，()代表分组，?P<NAME>为组命名
token_or = r'(?P<OR>\|\|)'
token_not = r'(?P<NOT>\!)'
token_word = r'(?P<WORD>[a-zA-Z]+)'
token_and = r'(?P<AND>&&)'
token_lp = r'(?P<LP>\()'
token_rp = r'(?P<RP>\))'
lexer = re.compile('|'.join([token_or, token_not, token_word,
                            token_and, token_lp, token_rp]))  # 编译正则表达式

In [219]:
# 用编译好的正则表达式进行词法分析
def get_tokens(query):
    """
    [('abc', 'WORD'), ('(', 'LP'), ('ab', 'WORD'), (')', 'RP'), ('hhh', 'WORD'), ('aa', 'WORD')]
    :param query: 
    :return: 
    """
    tokens = []  # tokens中的元素类型为(token, token类型)
    for token in re.finditer(lexer, query):
        tokens.append((token.group(), token.lastgroup))
    return tokens

## 布尔检索类
由构建索引、布尔表达式解析、结果查询与合并三部分组成

In [220]:
class BoolRetrieval:
    """
    布尔检索类
    index为字典类型，其键为单词，值为文件ID列表，如{"word": [1, 2, 9], ...}
    self.query_tokens为需要查询的词序列，list
    self.files 文件名列表
    """

    def __init__(self, index_path=''):
        '''

        :param index_path:
        '''
        if index_path == '':
            self.index = defaultdict(list)
        # 已有构建好的索引文件
        else:
            data = np.load(index_path, allow_pickle=True)
            self.files = data['files'][()]  # 没有很理解这后半部分【】的作用
            self.index = data['index'][()]
        self.query_tokens = []

    def build_index(self, text_dir):
        '''
        self.index就像是一个列表，索引键为词，索引值为所在文档的序号
        :param text_dir:
        :return:
        '''
        self.files = get_files(text_dir)  # 获取所有文件名
        for num in range(0, len(self.files)):
            f = open(self.files[num])
            text = f.read()
            words = get_words(text)  # 分词
            # 构建倒排索引
            for word in words:
                self.index[word].append(num)
        print(self.files, self.index)
        np.savez('index.npz', files=self.files, index=self.index)

    def search(self, query):
        self.query_tokens = get_tokens(query)  # 获取查询的tokens
        result = []
        # 将查询得到的文件ID转换成文件名
        for num in self.evaluate(0, len(self.query_tokens) - 1):
            result.append(self.files[num])
        return result

    # 递归解析布尔表达式，p、q为子表达式左右边界的下标
    def evaluate(self, p, q):
        # 解析错误
        if p > q:
            return []
        # 单个token，一定为查询词
        elif p == q:
            return self.index[self.query_tokens[p][0]]
        # 去掉外层括号
        elif self.check_parentheses(p, q):
            return self.evaluate(p + 1, q - 1)
        # else:
        #     # 非单个token
        #     op,and_list = self.find_operator(p, q)
        #     #只有一个and运算符
        #     if op == -1:
        #         return []
        #     # files1为运算符左边得到的结果，files2为右边
        #     if self.query_tokens[op][1] == 'NOT':
        #         files1 = []
        #     else:
        #         files1 = self.evaluate(p, op - 1)
        #     files2 = self.evaluate(op + 1, q)
        #     return self.merge(files1, files2, self.query_tokens[op][1])

        else:
            # 非单个token
            op, and_list = self.find_operator(p, q)
            # 只有一个and运算符
            if len(and_list) == 1 or not and_list:
                if op == -1:
                    return []
                # files1为运算符左边得到的结果，files2为右边
                if self.query_tokens[op][1] == 'NOT':
                    files1 = []
                else:
                    files1 = self.evaluate(p, op - 1)
                files2 = self.evaluate(op + 1, q)
                return self.merge(files1, files2, self.query_tokens[op][1])
            else:
                if op == -1:
                    return []
                # files1为运算符左边得到的结果，files2为右边
                if self.query_tokens[op][1] == 'NOT':
                    files1 = []
                elif self.query_tokens[op][1] == 'AND':  # 最小优先级为and，且有多个and

                    # 将p和q加入列表，方便统一格式
                    and_list.insert(0, p - 1)
                    and_list.insert(len(and_list), q + 1)
                    num_and = len(and_list)
                    files = []
                    # files.append(len(self.evaluate(p,and_list[0]-1)))
                    i = 0
                    while i < num_and - 1:
                        files.append(len(self.evaluate(and_list[i] + 1, and_list[i + 1] - 1)))
                        i += 1
                    # files.append(len(self.evaluate(and_list[-1]+1,q)))
                    print('各个段出现文章的个数分别为：',files)
                    temp = pd.Series(files).sort_values()[:2].values
                    print('即将进行运算的是以下两个段：',temp)
                    files1 = self.evaluate(and_list[temp[0]] + 1, and_list[temp[0] + 1] - 1)
                    files2 = self.evaluate(and_list[temp[1]] + 1, and_list[temp[1] + 1] - 1)
                    return self.merge(files1, files2, self.query_tokens[op][1])


                else:
                    files1 = self.evaluate(p, op - 1)
                files2 = self.evaluate(op + 1, q)
                return self.merge(files1, files2, self.query_tokens[op][1])

    # 判断表达式是否为 (expr)
    # 判断表达式是否为 (expr)
    def check_parentheses(self, p, q):
        """
        判断表达式是否为 (expr)
        整个表达式的左右括号必须匹配才为合法的表达式
        返回True或False

        #注意(based!day)&&(based&&day)
        """
        LP = 0
        if (self.query_tokens[p][0] == '(') & (self.query_tokens[q][0] == ')'):
            i = p + 1
            while (i < q):
                if self.query_tokens[i][0] == ')':
                    LP -= 1
                    if LP < 0: return False
                elif self.query_tokens[i][0] == '(':
                    LP += 1
                i += 1
            return True
        return False

    # 寻找表达式的dominant的运算符（优先级最低）
    def find_operator(self, p, q):
        """
        寻找表达式的dominant的运算符（优先级最低）
        其必定在括号外面（不存在整个子表达式被括号包围，前面以已处理）
        返回dominant运算符的下标位置
        """
        # 找出第一个非括号运算符，并将其位置记录在列表中
        LP = 0
        lowest_op = 'NOT'
        first_and = -1
        first_not = -1
        and_list = []
        while p < q:
            temp_token = self.query_tokens[p][1]
            if temp_token != 'WORD':
                if temp_token == 'LP':
                    LP += 1
                elif temp_token == 'RP':
                    LP -= 1
                elif LP == 0:
                    if temp_token == 'OR':
                        return p, []
                    elif temp_token == 'AND':
                        lowest_op = 'AND'
                        if first_and == -1: first_and = p
                        and_list.append(p)
                    elif temp_token == 'NOT':
                        if first_not == -1: first_not = p

            p += 1
        if lowest_op == 'AND':
            return first_and, and_list
        else:
            return first_not, []

    def merge(self, files1, files2, op_type):
        """
        根据运算符对进行相应的操作
        在Python中可以通过集合的操作来实现
        但为了练习算法，请遍历files1, files2合并
        """
        result = []

        if op_type == 'AND':
            # result = list(set(files1) & set(files2))
            result = [item for item in files1 if item in files2]
        elif op_type == "OR":
            # result = list(set(files1) | set(files2))
            temp_file_list = files1 + files2
            if temp_file_list:
                result = pd.array(temp_file_list).unique().to_numpy().tolist()
        elif op_type == "NOT":
            # result = list(set(range(0, len(self.files))) - set(files2))
            # all_files=np.array()

            result = [item for item in range(0, len(self.files)) if item not in files2]

        return result

## 创建布尔检索类对象
第一次需要调用build_index()函数创建索引，之后可直接用索引文件进行初始化

In [221]:
# br = BoolRetrieval()
# br.build_index('text')
br = BoolRetrieval('index.npz')
br.files

array(['text\\advantages.txt', 'text\\bir.txt', 'text\\disadvantage.txt',
       'text\\ir.txt'], dtype='<U21')

In [222]:
br.index

defaultdict(list,
            {'formalism': [0],
             'advantages': [0],
             'intuitive': [0],
             'clean': [0],
             'easy': [0],
             'concept': [0],
             'implement': [0],
             'terms': [1, 2],
             'conceived': [1],
             'many': [1, 2],
             'information': [1, 2, 3],
             'used': [1],
             'boolean': [1, 2],
             'logic': [1],
             'contain': [1],
             'theory': [1],
             'sets': [1],
             'retrieval': [1, 2, 3],
             'time': [1],
             'whether': [1],
             'first': [1],
             'searched': [1],
             'ir': [1, 3],
             'standard': [1],
             'one': [1],
             'bir': [1],
             'set': [1],
             'based': [1, 2, 3],
             'day': [1],
             'documents': [1, 2, 3],
             'classical': [1],
             'query': [1, 2],
             'model': [1, 2],
           


'boolean': [1, 2],

'based': [1, 2, 3],

 'day': [1],

In [226]:
# 'boolean': [1, 2],
# 
# 'based': [1, 2, 3],
# 
#  'day': [1],
# query = input("请输入与查询（或||，与&&，非！）：")
print(br.search('based&&!day'))
print(br.search('based&&day'))
print(br.search('day||boolean'))
print(br.search('(based&&!day)&&(based&&day)'))
print(br.search('(based&&!day)||(based&&day)'))
print(br.search('((based&&!day)||boolean)||(based&&day)'))
print('\n以下测试多个and的优化：')
print(br.search('based&&day&&boolean'))

['text\\disadvantage.txt', 'text\\ir.txt']
['text\\bir.txt']
['text\\bir.txt', 'text\\disadvantage.txt']
[]
['text\\disadvantage.txt', 'text\\ir.txt', 'text\\bir.txt']
['text\\disadvantage.txt', 'text\\ir.txt', 'text\\bir.txt']

以下测试多个and的优化：
各个段出现文章的个数分别为： [3, 1, 2]
即将进行运算的是以下两个段： [1 2]
['text\\bir.txt']
